In [6]:
import numpy as np
import pandas as pd
from Utils import Reader
URM_all_dataframe = pd.read_csv(filepath_or_buffer="../data/interactions_and_impressions_v2.csv",
                                 sep=",",
                                 skiprows=1,
                                 header=None,
                                 dtype={0: int, 1: int, 2: str, 3: float},
                                 engine='python')
URM_all_dataframe=URM_all_dataframe.drop(columns=[2],axis=1)
Reader.save(URM_all_dataframe,name="oo")

In [ ]:
# data processing, CSV file I/O (e.g. pd.read_csv)
URM_path="../data/interactions_and_impressions.csv"
URM_path2="../data/interactions_and_impressions_v3.csv"
URM_all_dataframe = pd.read_csv(URM_path)

URM_all_dataframe.columns = ["UserID", "ItemID", "others","Data"]
URM_all_dataframe=URM_all_dataframe.drop("others",axis=1)
URM_all_dataframe["Data"] = URM_all_dataframe["Data"].replace({0: 1, 1: 0})


URM_all_dataframe2 = pd.read_csv(URM_path2)
URM_all_dataframe2.columns = ["UserID", "ItemID","Data"]
#URM_all_dataframe2 = URM_all_dataframe2[URM_all_dataframe2['Data'] != 0.01]

URM_all_dataframe2 = URM_all_dataframe2.sort_values(by=['UserID', 'ItemID', 'Data'])
URM_all_dataframe = URM_all_dataframe.sort_values(by=['UserID', 'ItemID', 'Data'])

all_users_items=list(set(list(zip(URM_all_dataframe['UserID'],URM_all_dataframe['ItemID']))))
np.random.shuffle(all_users_items)


import random
train_percentage=0.8
user_for_training=random.sample(all_users_items, round(len(all_users_items)*train_percentage))
user_for_validation=list(set(all_users_items)-set(user_for_training))
user_for_training=pd.DataFrame(user_for_training,columns=["UserID","ItemID"])
user_for_validation=pd.DataFrame(user_for_validation,columns=["UserID","ItemID"])
data_train2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
data_train = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_training.set_index(["UserID","ItemID"]).index)]
#https://stackoverflow.com/questions/54006298/select-rows-of-a-dataframe-based-on-another-dataframe-in-python



data_valid2 = URM_all_dataframe2[URM_all_dataframe2.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]
data_valid = URM_all_dataframe[URM_all_dataframe.set_index(["UserID","ItemID"]).index.isin(user_for_validation.set_index(["UserID","ItemID"]).index)]

import numpy as np
import scipy.sparse as sps


URM_train = sps.csr_matrix((data_train["Data"].values,
                          (data_train["UserID"].values, data_train["ItemID"].values)))
URM_train2 = sps.csr_matrix((data_train2["Data"].values,
                          (data_train2["UserID"].values, data_train2["ItemID"].values)))
URM_valid = sps.csr_matrix((data_valid["Data"].values,
                             (data_valid["UserID"].values, data_valid["ItemID"].values)))
URM_valid2 = sps.csr_matrix((data_valid2["Data"].values,
                             (data_valid2["UserID"].values, data_valid2["ItemID"].values)))
URM_train,URM_train2 ,URM_valid,URM_valid2

In [1]:
from Utils import Reader
from Utils.Evaluator import EvaluatorHoldout
URM_train,URM_train2 ,URM_valid,URM_valid2=Reader.split_train_validation_double()
evaluator_validation=EvaluatorHoldout(URM_valid,cutoff_list=[10])
evaluator_validation2=EvaluatorHoldout(URM_valid2,cutoff_list=[10])


EvaluatorHoldout: Ignoring 1534 ( 3.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1494 ( 3.6%) Users that have less than 1 test interactions


In [2]:
#1
# BEST MODEL UNTIL NOW _ PROVA

from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

Similarity column 24507 (100.0%), 5478.67 column/sec. Elapsed time 4.47 sec
Similarity column 41629 (100.0%), 2233.07 column/sec. Elapsed time 18.64 sec
[[-0.22783144 -0.24661598 -0.14636414 ... -0.32209316 -0.1526905
  -0.31113636]
 [-0.31692392 -0.3441573  -0.2756629  ... -0.3461435  -0.25276658
  -0.3461435 ]
 [ 0.09845128  0.09323618  0.74743265 ... -0.11062562 -0.03308582
  -0.27279887]
 ...
 [ 0.31360623  0.31388116  0.4426419  ... -0.34195384 -0.03812265
  -0.2928145 ]
 [-0.3461435  -0.3461435  -0.3461435  ... -0.3461435  -0.3461435
  -0.3461435 ]
 [ 0.24755417 -0.12761603 -0.18620004 ... -0.34338626 -0.3461435
  -0.3461435 ]]
[[-0.04577835 -0.02905316  0.01731892 ... -0.04577835  0.0068675
  -0.04577835]
 [-0.04577835 -0.04577835 -0.04577835 ... -0.04577835 -0.04577835
  -0.04577835]
 [-0.03983438 -0.04577835  0.01799051 ... -0.03503622 -0.02498031
  -0.04577835]
 ...
 [-0.03160794 -0.03846579 -0.03152359 ... -0.04577835 -0.04577835
  -0.04577835]
 [-0.04577835 -0.04577835 -0.0

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[-0.17454287 -0.06733553 -0.03467036 ... -0.39272574 -0.39272574
  -0.3119317 ]
 [-0.39272574 -0.34359357 -0.38611513 ... -0.18964039 -0.36839673
  -0.39272574]
 [ 1.3125235  -0.3099215   0.3011348  ... -0.32530925 -0.29614773
   0.56753653]
 ...
 [-0.1412377  -0.21402363 -0.08106093 ... -0.32868907 -0.3244399
  -0.37107956]
 [-0.25272638 -0.30671623 -0.12024572 ... -0.39272574 -0.39272574
  -0.39272574]
 [-0.07963905 -0.19353694  0.11289194 ... -0.35158828 -0.27608067
   0.41930696]]
[[-0.03751625 -0.02987532 -0.03425322 ... -0.04725246 -0.04725246
  -0.04725246]
 [-0.04725246 -0.04725246 -0.04725246 ... -0.0283676  -0.03957834
  -0.04725246]
 [-0.03137992 -0.04650472 -0.04561702 ... -0.04523415 -0.04725246
  -0.03046078]
 ...
 [-0.01398073 -0.02510724 -0.02669848 ... -0.03960915 -0.04725246
  -0.04725246]
 [-0.03460702 -0.04590734 -0.03652434 ... -0.04725246 -0.04725246
  -0.04725246]
 [-0.03501983 -0.04604097 -0.0163286  ... -0.04725246 -0.04725246
  -0.00370258]]
[[ 0.26117802 -0.

KeyboardInterrupt: 

In [ ]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

In [ ]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

In [ ]:
#1
# BEST MODEL UNTIL NOW - CONTROPROVA
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(URM_train2)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation2.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

In [1]:
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
#URM_a          ll=Reader.load_URM(values_to_replace={0.01:0.02})
URM_path2="../output/more_than_10.csv"
_,URM_train ,URM_valid,URM_valid2=Reader.split_train_validation_double(URM_path2=URM_path2)
evaluator_test=EvaluatorHoldout(URM_valid,cutoff_list=[10])

EvaluatorHoldout: Ignoring 1581 ( 3.8%) Users that have less than 1 test interactions


In [12]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

r = SLIMElasticNetRecommender(URM_train)
r.fit()

result_df, _ = evaluator_test.evaluateRecommender(r)
result_df

SLIMElasticNetRecommender: URM Detected 95 ( 0.2%) users with no interactions.
SLIMElasticNetRecommender: Processed 24507 (100.0%) in 2.66 min. Items per second: 153.71
EvaluatorHoldout: Processed 40094 (100.0%) in 21.55 sec. Users per second: 1861


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.002566,0.005272,0.005087,0.001875,0.003799,0.017302,3157079069977797114011695002829363679083957104...,0.003412,0.023994,0.018011,...,0.963127,0.023109,0.963127,0.000563,4.040509,0.926196,0.001179,0.300767,0.626876,0.908444


In [13]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

r = SLIM_BPR_Cython(URM_train)
#r.fit(topK= 482, epochs= 110, symmetric= True, sgd_mode='adam', lambda_i= 0.01, lambda_j= 1e-05, learning_rate= 0.000362945311633336, positive_threshold_BPR=0.0)
r.fit(topK= 482, epochs= 110, symmetric= True, sgd_mode='adam', lambda_i= 0.01, lambda_j= 1e-05, learning_rate= 0.000362945311633336, positive_threshold_BPR=0.0)

result_df, _ = evaluator_test.evaluateRecommender(r)
result_df

SLIM_BPR_Recommender: URM Detected 95 ( 0.2%) users with no interactions.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.


/bin/sh: free: command not found


Processed 41629 (100.0%) in 0.71 sec. BPR loss is 1.30E-05. Sample per second: 58973
SLIM_BPR_Recommender: Epoch 1 of 110. Elapsed time 0.58 sec
Processed 41629 (100.0%) in 1.00 sec. BPR loss is 8.84E-05. Sample per second: 41509
SLIM_BPR_Recommender: Epoch 2 of 110. Elapsed time 0.87 sec
Processed 41629 (100.0%) in 0.24 sec. BPR loss is 2.19E-04. Sample per second: 173223
SLIM_BPR_Recommender: Epoch 3 of 110. Elapsed time 1.11 sec
Processed 41629 (100.0%) in 0.50 sec. BPR loss is 4.25E-04. Sample per second: 83383
SLIM_BPR_Recommender: Epoch 4 of 110. Elapsed time 1.37 sec
Processed 41629 (100.0%) in 0.67 sec. BPR loss is 7.06E-04. Sample per second: 61716
SLIM_BPR_Recommender: Epoch 5 of 110. Elapsed time 1.54 sec
Processed 41629 (100.0%) in 0.82 sec. BPR loss is 1.06E-03. Sample per second: 50625
SLIM_BPR_Recommender: Epoch 6 of 110. Elapsed time 1.69 sec
Processed 41629 (100.0%) in 0.95 sec. BPR loss is 1.43E-03. Sample per second: 43610
SLIM_BPR_Recommender: Epoch 7 of 110. Elapse

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


EvaluatorHoldout: Processed 40094 (100.0%) in 25.05 sec. Users per second: 1601


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.017491,0.03479,0.033478,0.007835,0.015503,0.060694,inf,0.022977,0.134833,0.068724,...,0.963127,0.129861,0.963127,0.207799,10.232102,0.989079,0.435357,0.761655,5.664938,0.67065


In [2]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
URM_all = Reader.read_train_csr(matrix_path="../data/interactions_and_impressions.csv",values_to_replace={0:0.3})
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all ,train_percentage=0.80)
evaluator_test=EvaluatorHoldout(URM_valid,cutoff_list=[10])

r = IALSRecommender(URM_train)
r.fit(num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
#r.fit()

result_df, _ = evaluator_test.evaluateRecommender(r)
result_df

KeyboardInterrupt: 

In [1]:
import Utils.Writer as Writer
from Utils.Writer import NameRecommender
import Utils.Reader as Reader
URM_all = Reader.read_train_csr(matrix_path="../data/interactions_and_impressions.csv",values_to_replace={0:0.3})

#URM_all_original=Reader.read_train_csr()
submit= Writer.Writer(NameRecommender=NameRecommender.IALS, URM=URM_all)
submit.makeSubmission()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
IALSRecommender: Epoch 1 of 115. Elapsed time 37.18 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 1.25 min
IALSRecommender: Epoch 3 of 115. Elapsed time 1.79 min
IALSRecommender: Epoch 4 of 115. Elapsed time 2.25 min
IALSRecommender: Epoch 5 of 115. Elapsed time 2.51 min
IALSRecommender: Epoch 6 of 115. Elapsed time 2.77 min
IALSRecommender: Epoch 7 of 115. Elapsed time 3.02 min
IALSRecommender: Epoch 8 of 115. Elapsed time 3.28 min
IALSRecommender: Epoch 9 of 115. Elapsed time 3.54 min
IALSRecommender: Epoch 10 of 115. Elapsed time 3.79 min
IALSRecommender: Epoch 11 of 115. Elapsed time 4.05 min
IALSRecommender: Epoch 12 of 115. Elapsed time 4.30 min
IALSRecommender: Epoch 13 of 115. Elapsed time 4.55 min
IALSRecommender: Epoch 14 of 115. Elapsed time 4.81 min
IALSRecommender: Epoch 15 of 115. Elapsed time 5.06 min
IALSRecommender: Epoch 16 of 115. Elapsed time 5.31 min
IALSRecommender: Epoch 17 

In [15]:
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender

recommender_ItemUserHybridKNNRecommender = ItemUserHybridKNNRecommender(URM_train)
recommender_ItemUserHybridKNNRecommender.fit()

result_df, _ = evaluator_test.evaluateRecommender(recommender_ItemUserHybridKNNRecommender)
result_df

ItemUserHybridKNNRecommender: URM Detected 95 ( 0.2%) users with no interactions.
ItemKNNCFRecommender: URM Detected 95 ( 0.2%) users with no interactions.
UserKNNCFRecommender: URM Detected 95 ( 0.2%) users with no interactions.
Similarity column 24507 (100.0%), 6382.07 column/sec. Elapsed time 3.84 sec
Similarity column 41629 (100.0%), 3626.45 column/sec. Elapsed time 11.48 sec
[[ 0.06819762  1.4982314   0.59135276 ... -0.12218025 -0.20868279
  -0.22577198]
 [-0.22577198 -0.1707559  -0.22577198 ... -0.22577198 -0.22577198
  -0.22577198]
 [ 0.84949505  3.3451388   1.0114602  ... -0.22577198 -0.06689065
  -0.13850752]
 ...
 [-0.1610207  -0.22577198 -0.22577198 ... -0.22577198 -0.22577198
  -0.22577198]
 [ 0.27836066  1.2530501   0.10964005 ... -0.22577198 -0.22577198
  -0.12284812]
 [-0.20339456 -0.22577198 -0.22577198 ... -0.17679025 -0.22577198
  -0.22577198]]
[[-6.28306717e-03  3.26836458e-03  3.96002769e+00 ... -8.81795585e-03
  -1.52413845e-02 -1.52413845e-02]
 [-1.52413845e-02 -1

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),


[[-0.23918334 -0.23918334 -0.23918334 ... -0.23918334 -0.23918334
  -0.23918334]
 [ 0.21446519  0.98677266  0.1730503  ... -0.23918334 -0.07945648
  -0.23918334]
 [-0.23918334 -0.23918334 -0.23918334 ... -0.23918334 -0.23918334
   0.16393815]
 ...
 [ 0.05029675  0.29880142 -0.008105   ... -0.23918334 -0.23918334
  -0.23918334]
 [-0.19171876 -0.23918334 -0.23918334 ... -0.23918334 -0.23918334
  -0.23918334]
 [ 0.42810515  0.28173104  0.73485523 ...  0.4395366   0.22785878
  -0.13852893]]
[[-0.02966105 -0.02966105 -0.02966105 ... -0.02966105 -0.02966105
  -0.02966105]
 [-0.02141549  0.00740624 -0.02236385 ... -0.02966105 -0.01218366
  -0.02966105]
 [-0.02966105 -0.02966105 -0.02966105 ... -0.02966105 -0.02966105
  -0.01087466]
 ...
 [ 0.00088763 -0.00835479 -0.02966105 ... -0.02966105 -0.02966105
  -0.02966105]
 [-0.02966105 -0.02966105 -0.02966105 ... -0.02966105 -0.02966105
  -0.02966105]
 [ 0.05658921  0.04836053  0.10853578 ...  0.01445064 -0.00781313
  -0.00373009]]
[[-0.18514936  0

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.020769,0.03982,0.038065,0.009751,0.018583,0.070555,inf,0.026874,0.148626,0.082479,...,0.963127,0.143145,0.963127,0.061791,10.340929,0.996997,0.129457,0.769756,3.53592,0.68637


In [16]:
from collections import Counter
l=[]
l.append((0,0,1))
l.append((0,0,1))
l.append((0,0,1))
l.append((0,0,1))
l.append((0,0,2))
l.append((0,0,3))
l.append((0,0,3))
l.append((0,0,3))
l= list({k: v for k, v in Counter(l).items() if v >= 4}.keys())

In [17]:
l

[(0, 0, 1)]

In [2]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender


In [11]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender

URM_all=Reader.read_train_csr(values_to_replace={0:0.03})

#recommender_ItemUserHybridKNNRecommender = ItemUserHybridKNNRecommender(URM_all)
#recommender_ItemUserHybridKNNRecommender.fit()
recommender_ItemKNNCF = ItemKNNCFRecommender(URM_all)
recommender_ItemKNNCF.fit(
)
ials=IALSRecommender(URM_all)
ials.fit( num_factors= 77, epochs= 115, confidence_scaling= 'linear', alpha=  3.951833188581218, epsilon= 1.1320191944038327, reg= 4.746687709282205e-05)
import scipy.sparse as sps
scoreshybridrecommender = ScoresHybridRecommender(URM_all, recommender_ItemKNNCF, ials)
scoreshybridrecommender.fit(alpha = 0.078)
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:
    d.write("user_id,item_list\n")
    writer = csv.writer(d, delimiter=' ')

    for i in range(len(user_array)):
        d.write(str(user_array[i]) + ", ")
        #             if(self.NameRecommender.name!="Rankings"):
        recommandations = scoreshybridrecommender.recommend(user_id_array=user_array[i], cutoff=10)
        #               else:
        #                    recommandations=self.Recommender.recomendation_ranking(user_id_array=user_array[i])
        array = np.asarray(recommandations)
        writer.writerow(array)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
Similarity column 24507 (100.0%), 4098.12 column/sec. Elapsed time 5.98 sec
IALSRecommender: Epoch 1 of 115. Elapsed time 17.61 sec
IALSRecommender: Epoch 2 of 115. Elapsed time 37.33 sec
IALSRecommender: Epoch 3 of 115. Elapsed time 55.08 sec
IALSRecommender: Epoch 4 of 115. Elapsed time 1.21 min
IALSRecommender: Epoch 5 of 115. Elapsed time 1.49 min
IALSRecommender: Epoch 6 of 115. Elapsed time 1.75 min
IALSRecommender: Epoch 7 of 115. Elapsed time 2.01 min
IALSRecommender: Epoch 8 of 115. Elapsed time 2.26 min
IALSRecommender: Epoch 9 of 115. Elapsed time 2.52 min
IALSRecommender: Epoch 10 of 115. Elapsed time 2.83 min
IALSRecommender: Epoch 11 of 115. Elapsed time 3.08 min
IALSRecommender: Epoch 12 of 115. Elapsed time 3.35 min
IALSRecommender: Epoch 13 of 115. Elapsed time 3.61 min
IALSRecommender: Epoch 14 of 115. Elapsed time 3.99 min
IALSRecommender: Epoch 15 of 115. Elapsed time 4.29 min
IALS

NameError: name 'sps' is not defined

In [14]:
import scipy.sparse as sps
scoreshybridrecommender = ScoresHybridRecommender(URM_all, recommender_ItemKNNCF, ials)
scoreshybridrecommender.fit(alpha = 0.078)
import os
import numpy as np
import csv
import pandas as pd
parent_dir =  "../"
read_path = os.path.join(parent_dir, "data/data_target_users_test.csv")

with open(read_path, "r") as f:
    dataFrame = pd.read_csv(f, skiprows=0)
    dataFrame.columns = ["UserID"]
    user_array = np.asarray(dataFrame["UserID"])

write_path = os.path.join(parent_dir, "Testing_Results/submission.csv")
print(user_array)
user_array = user_array.tolist()
with open(write_path, "w+") as d:
    d.write("user_id,item_list\n")
    writer = csv.writer(d, delimiter=' ')

    for i in range(len(user_array)):
        d.write(str(user_array[i]) + ", ")
        #             if(self.NameRecommender.name!="Rankings"):
        recommandations = scoreshybridrecommender.recommend(user_id_array=user_array[i], cutoff=10)
        #               else:
        #                    recommandations=self.Recommender.recomendation_ranking(user_id_array=user_array[i])
        array = np.asarray(recommandations)
        writer.writerow(array)

[    0     1     2 ... 41626 41627 41628]


In [13]:
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2


    def fit(self, alpha = 0.5):
        self.alpha = alpha


    def _compute_item_score(self, user_id_array, items_to_compute):

        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights